In [1]:
import keras
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from keras.layers import Dense

from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

%matplotlib inline

Using TensorFlow backend.


In [2]:
from numpy import genfromtxt
EEG = genfromtxt("C:/Users/SoobinYim/workspace/Confusion during MOOC/EEG_data.csv", delimiter=",")

In [3]:
#6의 raw데이터에 이상이 있으므로 6을 제외하고 분석
EEG=EEG[1:,:]
EEG=pd.DataFrame(EEG)
remove_6=EEG[EEG[0]==6].index
print(EEG.shape, remove_6.shape)
EEG=EEG.drop(remove_6)
print(EEG.shape)
EEG=EEG.values

(12811, 15) (1275,)
(11536, 15)


In [4]:
X=EEG
y = EEG[:, -1]
print(X.shape,y.shape)

(11536, 15) (11536,)


In [5]:
#동영상, raw, 주파수 data 사용
X1=pd.DataFrame(X)
X1=X1.drop(X1.columns[[0,2,3,13,14]], axis='columns')
X1.head()
#사람 정보, raw, 주파수 data 사용
X2=pd.DataFrame(X)
X2=X2.drop(X2.columns[[1,2,3,13,14]], axis='columns')
X2.head()
#동영상, raw, 주파수 data 사용
X3=pd.DataFrame(X)
X3=X3.drop(X3.columns[[0,2,3,4,13,14]], axis='columns')
X3.head()
#사람 정보, 주파수 data 사용
X4=pd.DataFrame(X)
X4=X4.drop(X4.columns[[1,2,3,4,13,14]], axis='columns')
X4.head()

X1=X1.values
X2=X2.values
X3=X3.values
X4=X4.values

In [88]:
#X=X1
#X=X2
X=X3
#X=X4

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=14)

In [73]:
from keras.models import Sequential, Model 
from keras.layers import Dense, Conv2D, Convolution2D, MaxPooling2D, Dropout, Flatten, TimeDistributed, InputLayer, LSTM
from keras.layers import Input, Reshape, Activation, add, Add
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

In [90]:
from tensorflow.keras import utils as np_utils
#one-hot encoding
y_train      = np_utils.to_categorical(y_train)
y_test       = np_utils.to_categorical(y_test)

print(y_train.shape, y_test.shape)

(8075, 2) (3461, 2)


In [91]:
X_train = X_train.reshape(X_train.shape+ (1,))
X_test = X_test.reshape(X_test.shape + (1,))

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8075, 9, 1)
(3461, 9, 1)
(8075, 2)
(3461, 2)


In [93]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape = (9,1), return_sequences = True))
    model.add(LSTM(2, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = lstm_model()

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 9, 50)             10400     
_________________________________________________________________
lstm_16 (LSTM)               (None, 2)                 424       
_________________________________________________________________
activation_8 (Activation)    (None, 2)                 0         
Total params: 10,824
Trainable params: 10,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist=model.fit(X_train, y_train, epochs = 100, batch_size = 50, verbose = 0, validation_data=(X_test,y_test))

In [ ]:
%matplotlib inline

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

results = model.evaluate(X_test, y_test)
print('Test accuracy: ', results[1])